In [39]:
import cv2
import numpy as np

In [40]:
# crop video to length. Do not redo this.
from cs585_common import crop_video

input_video_path = 'data/audi_raw_data.mp4'
output_video_path = 'data/audi_gravel_road_footage.mp4'
start_time = (5 * 60)  # seconds
end_time = (8 * 60 + 30)   # seconds

#crop_video(input_video_path, output_video_path, start_time, end_time)

In [41]:
def crop_road_snippet(frame):
    # consts found by trial and error
    return frame[100:200, 500:800]

def crop_steering_wheel(frame):
    return frame[300:850, 300:925]

In [42]:
def segment_gravel_road(frame):
    # convert to HSV
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    blurred_frame = cv2.GaussianBlur(hsv_frame, (11, 11), 0)

    lower_gray = np.array([0, 0, 50])
    upper_gray = np.array([179, 50, 220])

    mask = cv2.inRange(blurred_frame, lower_gray, upper_gray)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour with the largest area
    largest_contour = max(contours, key=cv2.contourArea)

    # Create a mask for the largest contour
    output = np.zeros(mask.shape)
    cv2.drawContours(output, [largest_contour], -1, 1, cv2.FILLED)

    return output

def segment_steering_wheel(frame):
    # convert to HSV
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #blurred_frame = cv2.GaussianBlur(rgb, (25, 25), 0)

    lower_gray = np.array([128, 128, 128])
    upper_gray = np.array([255, 255, 255])

    mask = cv2.inRange(rgb, lower_gray, upper_gray)

    circles = cv2.HoughCircles(grey, cv2.HOUGH_GRADIENT, 1.3, 30, param1=150, param2=70, minRadius=0, maxRadius=0)

    return mask, circles

def segment_hands(frame):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    blurred_frame = cv2.GaussianBlur(hsv_frame, (11, 11), 0)

    lower = np.array([0, 10, 60], dtype = "uint8") 
    upper = np.array([20, 150, 255], dtype = "uint8")

    mask = cv2.inRange(blurred_frame, lower, upper)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour with the largest area
    sorted_contours = sorted(contours, key=cv2.contourArea)

    hand1 = sorted_contours[-1]
    hand2 = sorted_contours[-2]

    def center_of_contours(c):
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])

        return (cX, cY)
    
    c1 = center_of_contours(hand1)
    c2 = center_of_contours(hand2)

    return (c1, c2)

In [46]:
# Open the video file
input_video_path = 'data/audi_gravel_road_footage.mp4'
video_capture = cv2.VideoCapture(input_video_path)

# Loop through frames
while True:
    # Read a frame from the video
    ret, frame = video_capture.read()

    # Check if the frame is successfully read
    if not ret:
        break

    # frame = crop_road_snippet(frame)
    # frame = segment_gravel_road(frame)

    frame = crop_steering_wheel(frame)
    c1, c2 = segment_hands(frame)
    
    cv2.line(frame, c1, c2, (0, 255, 0), 2)
    cv2.circle(frame, c1, 15, (255, 0, 0), -1)
    cv2.circle(frame, c2, 15, (255, 0, 0), -1)

    # Display the frame
    cv2.imshow('Frame', frame)

    # Wait for key press
    key = cv2.waitKey(0) & 0xFF

    # Quit if 'q' is pressed
    if key == ord('q'):
        break

# Release video capture
video_capture.release()
cv2.destroyAllWindows()